# week5 Spark Homework Quiz

### Quiz 1.
### Question 1. HVFHW February 2021
In this homework we'll put what we learned about Spark in practice.

We'll use high volume for-hire vehicles (HVFHV) dataset for that.

Download the HVFHV data for february 2021:

wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Read it with Spark using the same schema as we did in the lessons. We will use this dataset for all the remaining questions.

Repartition it to 24 partitions and save it to parquet.

* What's the size of the folder with results (in MB)?

In [3]:
#!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
!wget https://s3.amazonaws.com/nyc-tlc/csv_backup/fhvhv_tripdata_2021-02.csv    

--2022-07-05 20:53:23--  https://s3.amazonaws.com/nyc-tlc/csv_backup/fhvhv_tripdata_2021-02.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.197.176
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.197.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699.83M  2.51MB/s    in 7m 44s  

2022-07-05 21:01:08 (1.51 MB/s) - ‘fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [16]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, TimestampType, IntegerType, StructType, StructField

In [17]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-02.csv')


In [18]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [19]:
df.columns

['hvfhs_license_num',
 'dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag']

In [115]:
schema = StructType([
    StructField('hvfhs_license_num', StringType(), True),
    StructField('dispatching_base_num', StringType(), True),
    StructField('pickup_datetime', TimestampType(), True),
    StructField('dropoff_datetime', TimestampType(), True),
    StructField('PULocationID', IntegerType(), True),
    StructField('DOLocationID', IntegerType(), True),
    StructField('SR_Flag', StringType(), True)
])

In [119]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

In [120]:
df = df.repartition(24)

In [122]:
df.write.parquet('fhvhv_tripdata/2021/02/', mode='overwrite')

In [123]:
!du -sh fhvhv_tripdata/

210M	fhvhv_tripdata/


In [124]:
!du -ch  ~/data-engineering-zoomcamp/week_5_batch_processing/code/fhvhv_tripdata/2021/02/part-000*

8.7M	/home/thamizh/data-engineering-zoomcamp/week_5_batch_processing/code/fhvhv_tripdata/2021/02/part-00000-98d45f1b-31e7-4613-908e-92680769e48d-c000.snappy.parquet
8.7M	/home/thamizh/data-engineering-zoomcamp/week_5_batch_processing/code/fhvhv_tripdata/2021/02/part-00001-98d45f1b-31e7-4613-908e-92680769e48d-c000.snappy.parquet
8.7M	/home/thamizh/data-engineering-zoomcamp/week_5_batch_processing/code/fhvhv_tripdata/2021/02/part-00002-98d45f1b-31e7-4613-908e-92680769e48d-c000.snappy.parquet
8.7M	/home/thamizh/data-engineering-zoomcamp/week_5_batch_processing/code/fhvhv_tripdata/2021/02/part-00003-98d45f1b-31e7-4613-908e-92680769e48d-c000.snappy.parquet
8.7M	/home/thamizh/data-engineering-zoomcamp/week_5_batch_processing/code/fhvhv_tripdata/2021/02/part-00004-98d45f1b-31e7-4613-908e-92680769e48d-c000.snappy.parquet
8.7M	/home/thamizh/data-engineering-zoomcamp/week_5_batch_processing/code/fhvhv_tripdata/2021/02/part-00005-98d45f1b-31e7-4613-908e-92680769e48d-c000.snappy.parquet
8.7M	/home

* ANS: 208M

### Question 2. Count records

How many taxi trips were there on February 15?

Consider only trips that started on February 15.

In [125]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [126]:
from pyspark.sql import functions as F

In [33]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02867|2021-02-04 12:17:47|2021-02-04 12:34:48|         254|         212|   null|
|           HV0005|              B02510|2021-02-06 16:31:12|2021-02-06 16:55:40|          68|         141|   null|
|           HV0003|              B02872|2021-02-02 14:57:30|2021-02-02 15:08:19|         148|         211|   null|
|           HV0005|              B02510|2021-02-02 14:37:13|2021-02-02 14:44:43|         260|         129|   null|
|           HV0003|              B02617|2021-02-03 17:30:37|2021-02-03 17:40:59|          91|         149|   null|
|           HV0003|              B02866|2021-02-06 15:46:51|2021-02-06 16:00:59|

In [132]:
result_df = df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', df.dispatching_base_num) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID')
   
    


In [131]:
result_df

5

In [133]:
result_df.filter((result_df.pickup_date == '2021-02-15')).show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
| B02876| 2021-02-15|  2021-02-15|          74|          42|
| B02836| 2021-02-15|  2021-02-15|          72|          77|
| B02835| 2021-02-15|  2021-02-15|         234|         157|
| B02510| 2021-02-15|  2021-02-15|         116|         247|
| B02872| 2021-02-15|  2021-02-15|         132|          17|
| B02875| 2021-02-15|  2021-02-15|         255|           7|
| B02872| 2021-02-15|  2021-02-15|         244|         127|
| B02872| 2021-02-15|  2021-02-15|           3|         242|
| B02764| 2021-02-15|  2021-02-15|         230|         170|
| B02878| 2021-02-15|  2021-02-15|         243|         152|
| B02510| 2021-02-15|  2021-02-15|         175|         171|
| B02510| 2021-02-15|  2021-02-15|          17|          36|
| B02872| 2021-02-15|  2021-02-15|         215|         216|
| B02872| 2021-02-15|  2

In [134]:
result_df.filter((result_df.pickup_date == '2021-02-15')).count()

367170

In [40]:
result_df.filter((result_df.pickup_date == '2021-02-15')).count()

367170

### ANS: 367170

### Question 3. Longest trip for each day

Now calculate the duration for each trip.

Trip starting on which day was the longest?

In [148]:
result_df1 = df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', df.dispatching_base_num) \
    .withColumn('duration', (df.dropoff_datetime.cast("long") - df.pickup_datetime.cast("long"))/60 ) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'duration', 'PULocationID', 'DOLocationID')
    

In [149]:
result_df1.show()

+-------+-----------+------------+------------------+------------+------------+
|base_id|pickup_date|dropoff_date|          duration|PULocationID|DOLocationID|
+-------+-----------+------------+------------------+------------+------------+
| B02800| 2021-02-05|  2021-02-05|10.083333333333334|         161|         143|
| B02510| 2021-02-03|  2021-02-03|28.083333333333332|         246|          74|
| B02510| 2021-02-04|  2021-02-04|              28.5|          89|         181|
| B02682| 2021-02-04|  2021-02-04|13.733333333333333|         121|         219|
| B02879| 2021-02-02|  2021-02-02| 7.033333333333333|         153|         127|
| B02871| 2021-02-01|  2021-02-01| 33.63333333333333|         205|         177|
| B02872| 2021-02-01|  2021-02-01|              9.55|         244|         243|
| B02889| 2021-02-03|  2021-02-03| 7.633333333333334|          92|          92|
| B02510| 2021-02-04|  2021-02-04|23.883333333333333|         203|         265|
| B02510| 2021-02-04|  2021-02-04|12.716

In [153]:
result_df1.filter(result_df1.pickup_date == '2021-02-03').show()

+-------+-----------+------------+------------------+------------+------------+
|base_id|pickup_date|dropoff_date|          duration|PULocationID|DOLocationID|
+-------+-----------+------------+------------------+------------+------------+
| B02888| 2021-02-03|  2021-02-03|               9.3|         213|         183|
| B02888| 2021-02-03|  2021-02-03|             21.95|         141|         185|
| B02864| 2021-02-03|  2021-02-03|              14.8|          47|         185|
| B02876| 2021-02-03|  2021-02-03|12.083333333333334|         168|         213|
| B02510| 2021-02-03|  2021-02-03|             37.95|          75|         112|
| B02871| 2021-02-03|  2021-02-03|              18.1|          20|         235|
| B02764| 2021-02-03|  2021-02-03| 40.68333333333333|          10|          35|
| B02876| 2021-02-03|  2021-02-03|23.283333333333335|          95|         102|
| B02888| 2021-02-03|  2021-02-03| 8.283333333333333|           3|         254|
| B02510| 2021-02-03|  2021-02-03|16.633

In [145]:
result_df1.printSchemahemaSchemantSchema()

root
 |-- base_id: string (nullable = true)
 |-- pickup_date: date (nullable = true)
 |-- dropoff_date: date (nullable = true)
 |-- duration: interval (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)



In [79]:
result_df1.select('base_id', 'pickup_date', 'dropoff_date', 'duration').show()


+-------+-----------+------------+--------------------+
|base_id|pickup_date|dropoff_date|            duration|
+-------+-----------+------------+--------------------+
| B02864| 2021-02-03|  2021-02-03|24 minutes 31 sec...|
| B02395| 2021-02-02|  2021-02-02|6 minutes 18 seconds|
| B02870| 2021-02-06|  2021-02-06|3 minutes 37 seconds|
| B02887| 2021-02-04|  2021-02-04|8 minutes 37 seconds|
| B02510| 2021-02-04|  2021-02-05|22 minutes 42 sec...|
| B02872| 2021-02-03|  2021-02-03|11 minutes 36 sec...|
| B02872| 2021-02-03|  2021-02-03|25 minutes 38 sec...|
| B02866| 2021-02-06|  2021-02-06|11 minutes 36 sec...|
| B02875| 2021-02-04|  2021-02-04|12 minutes 16 sec...|
| B02872| 2021-02-05|  2021-02-05|7 minutes 27 seconds|
| B02510| 2021-02-05|  2021-02-05|30 minutes 12 sec...|
| B02872| 2021-02-03|  2021-02-03|45 minutes 32 sec...|
| B02882| 2021-02-06|  2021-02-06|21 minutes 12 sec...|
| B02872| 2021-02-02|  2021-02-02| 6 minutes 6 seconds|
| B02510| 2021-02-06|  2021-02-06|21 minutes 4 s

In [140]:
result_df1.printSchema()

root
 |-- base_id: string (nullable = true)
 |-- pickup_date: date (nullable = true)
 |-- dropoff_date: date (nullable = true)
 |-- duration: interval (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)



In [160]:
# df.agg(max(df(df.columns(1))), min(df(df.columns(1)))).show
# df.groupBy('pickup_date').max('duration').collect()

from pyspark.sql.functions import col
result_df2 = result_df1.groupBy('pickup_date').max('duration').collect()


In [165]:
result_df2

[Row(pickup_date=datetime.date(2021, 2, 15), max(duration)=431.23333333333335),
 Row(pickup_date=datetime.date(2021, 2, 2), max(duration)=515.2166666666667),
 Row(pickup_date=datetime.date(2021, 2, 26), max(duration)=407.03333333333336),
 Row(pickup_date=datetime.date(2021, 2, 21), max(duration)=537.05),
 Row(pickup_date=datetime.date(2021, 2, 5), max(duration)=508.51666666666665),
 Row(pickup_date=datetime.date(2021, 2, 10), max(duration)=569.4833333333333),
 Row(pickup_date=datetime.date(2021, 2, 1), max(duration)=343.96666666666664),
 Row(pickup_date=datetime.date(2021, 2, 6), max(duration)=524.1166666666667),
 Row(pickup_date=datetime.date(2021, 2, 19), max(duration)=626.2833333333333),
 Row(pickup_date=datetime.date(2021, 2, 20), max(duration)=733.9833333333333),
 Row(pickup_date=datetime.date(2021, 2, 8), max(duration)=501.76666666666665),
 Row(pickup_date=datetime.date(2021, 2, 9), max(duration)=534.7833333333333),
 Row(pickup_date=datetime.date(2021, 2, 11), max(duration)=1259.

ANS: Row(pickup_date=datetime.date(2021, 2, 11), max(duration)=1259.0),


### Question 4. Most common locations pair
Find the most common pickup-dropoff pair.

For example:

"Jamaica Bay / Clinton East"

Enter two zone names separated by a slash

If any of the zone names are unknown (missing), use "Unknown". For example, "Unknown / Clinton East".

In [167]:
df.registerTempTable('fhvhv_2021_02')

In [173]:
df_zones = spark.read.parquet('/home/thamizh/zones')

In [174]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [175]:
df_zones.registerTempTable('zones')

In [176]:
spark.sql("""
SELECT
    CONCAT(pul.Zone, ' / ', dol.Zone) AS pu_do_pair,
    COUNT(1)
FROM 
    fhvhv_2021_02 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
                      LEFT JOIN zones dol ON fhv.DOLocationID = dol.LocationID
GROUP BY 
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

+--------------------+--------+
|          pu_do_pair|count(1)|
+--------------------+--------+
|East New York / E...|   45041|
|Borough Park / Bo...|   37329|
| Canarsie / Canarsie|   28026|
|Crown Heights Nor...|   25976|
|Bay Ridge / Bay R...|   17934|
+--------------------+--------+

